# Decision Trees on Titanic (Classification)

Στόχοι του notebook:

- Να υλοποιήσουμε ένα δέντρο αποφάσεων για κατηγοριοποίηση (επιβίωση στο Titanic).
- Να συγκρίνουμε Gini vs Entropy (Information Gain).
- Να δούμε underfitting / overfitting μεταβάλλοντας το `max_depth`.
- Να κάνουμε απλό inference σε έναν νέο επιβάτη.

Αναμενόμενο αρχείο δεδομένων:

- `data/titanic_train.csv` (από το Kaggle Titanic competition).


In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree

ROOT = Path("..").resolve()
DATA_PATH = ROOT / "data" / "titanic_train.csv"

DATA_PATH


In [ ]:
df = pd.read_csv(DATA_PATH)
df.head()


## Επιλογή χαρακτηριστικών & προεπεξεργασία

- Χρησιμοποιούμε ένα υποσύνολο από τα διαθέσιμα χαρακτηριστικά.
- Κάνουμε απλή αντιμετώπιση ελλιπών τιμών (median / mode).
- Χρησιμοποιούμε `OneHotEncoder` για κατηγορικά χαρακτηριστικά.


In [ ]:
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
target = "Survived"

df = df[features + [target]]

df["Age"] = df["Age"].fillna(df["Age"].median())
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

X = df[features]
y = df[target]

cat_cols = ["Sex", "Embarked"]
num_cols = [c for c in features if c not in cat_cols]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ]
)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=0
)

X_train.shape, X_val.shape


## Εκπαίδευση δέντρου με διαφορετικά κριτήρια (Gini / Entropy)


In [ ]:
def train_and_eval(criterion, max_depth=None):
    clf = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        random_state=0,
    )
    pipe = Pipeline([("preprocess", preprocessor), ("model", clf)])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_val)
    print(f"=== criterion={criterion}, max_depth={max_depth} ===")
    print(classification_report(y_val, y_pred, digits=3))
    return pipe

pipe_gini = train_and_eval("gini", max_depth=4)
pipe_entropy = train_and_eval("entropy", max_depth=4)


## Underfitting / Overfitting: επίδραση του `max_depth`


In [ ]:
depths = [2, 3, 4, 6, 8, None]
rows = []

for d in depths:
    clf = DecisionTreeClassifier(criterion="entropy", max_depth=d, random_state=0)
    pipe = Pipeline([("preprocess", preprocessor), ("model", clf)])
    pipe.fit(X_train, y_train)
    acc_train = pipe.score(X_train, y_train)
    acc_val = pipe.score(X_val, y_val)
    rows.append({"max_depth": d if d is not None else "None",
                 "train_acc": acc_train,
                 "val_acc": acc_val})

results_df = pd.DataFrame(rows)
results_df


In [ ]:
plt.figure()
plt.plot(results_df["max_depth"], results_df["train_acc"], marker="o", label="train")
plt.plot(results_df["max_depth"], results_df["val_acc"], marker="o", label="val")
plt.xlabel("max_depth")
plt.ylabel("accuracy")
plt.title("Effect of tree depth (entropy criterion)")
plt.legend()
plt.show()


## Οπτικοποίηση δέντρου (μικρό βάθος)

Χρησιμοποιούμε το μοντέλο με `criterion="entropy"` και `max_depth=4` για να
δούμε τη δομή του δέντρου.


In [ ]:
tree_clf = pipe_entropy.named_steps["model"]
ohe = pipe_entropy.named_steps["preprocess"].named_transformers_["cat"]
cat_feature_names = list(ohe.get_feature_names_out(cat_cols))
feature_names = cat_feature_names + num_cols

plt.figure(figsize=(22, 10))
plot_tree(
    tree_clf,
    feature_names=feature_names,
    class_names=["Died", "Survived"],
    filled=True,
    rounded=True,
    fontsize=8,
)
plt.show()


## Inference για έναν νέο επιβάτη

Ορίζουμε έναν υποθετικό επιβάτη και ζητάμε από το μοντέλο να προβλέψει
αν θα επιβίωνε ή όχι.


In [ ]:
example = {
    "Pclass": 3,
    "Sex": "female",
    "Age": 22.0,
    "SibSp": 1,
    "Parch": 0,
    "Fare": 7.25,
    "Embarked": "S",
}

X_new = pd.DataFrame([example])
proba = pipe_entropy.predict_proba(X_new)[0]
pred = pipe_entropy.predict(X_new)[0]

print("Input:", example)
print("Predicted:", int(pred), "(0 = died, 1 = survived)")
print("Probabilities:", proba)
